In [1]:
"""This notebook is to optimize hyperparameters for LSTM Model For Regression using Randomized Search"""

# seperated by npast_timesteps to make preprocessing easier (tried doing it inside the wrapper and ran into many shape issues) 
# kernal crashed when I tried to do it all in one for loop

'This notebook is to optimize hyperparameters for LSTM Model using Randomized Search'

In [2]:
# imports 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nrsim_lstm import NRSIM_LSTM as lstm
import utils277b as utils

2024-12-11 22:04:45.441292: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 22:04:45.442366: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-11 22:04:45.453380: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-11 22:04:45.472460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733983485.493920  460378 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733983485.50

## Data Preprocessing and Set Parameters for LSTM

In [3]:
# load in data
data = pd.read_csv("test_file")
data.head()

,Time,pH,Hydrogen,Total Gas,Temperature,Pressure,Radioactivity,Power,Reactor Safety,Injection of Air,Injection of Air Degree,Resin Overheat,Resin Overheat Degree,Fuel Element Failure,Fuel Element Failure Degree,Chemical Addition,Vent Gas
0,0,11.000,50.000000,60.000000,500.000000,2100.000000,10.0,100.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,1,10.998,50.112998,60.112998,500.732703,2104.971916,10.0,100.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,2,10.996,50.225941,60.225941,501.463398,2109.930204,10.0,100.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,3,10.994,50.338517,60.338517,502.190083,2114.861274,10.0,100.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,4,10.992,50.450414,60.450414,502.910764,2119.751611,10.0,100.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [4]:
numerical = ['Time', 'pH', 'Hydrogen', 'Total Gas', 'Temperature', 'Pressure','Radioactivity', 'Power']
categorical = ['Reactor Safety', 'Injection of Air', 'Injection of Air Degree', 
               'Resin Overheat', 'Resin Overheat Degree', 'Fuel Element Failure', 
               'Fuel Element Failure Degree', 'Chemical Addition', 'Vent Gas']


# In case there are NaN values
data = data.fillna(False)
data[categorical] = data[categorical].astype(int) # set categorical as integer

# make sure no NaN values
unique_values = data.apply(pd.Series.unique)
# print(unique_values)

/tmp/ipykernel_460378/1598807462.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.fillna(False)


In [5]:
# training matrices
X = np.array(data[numerical].drop(columns=['Time']))
Y = np.array(data[categorical])

## Randomized Search - Regression

In [6]:
from sklearn.base import BaseEstimator, RegressorMixin
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

class PreprocessingWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, n_timesteps=1, n_features=7, neurons=[32, 32], activation='tanh', 
                 optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'],
                 dropout=0.2, conv_layer=False, nfilters=64, conv_act='relu', pool_size=2, 
                 classification=True, n_predicted_timesteps = 1, epochs = 50, batch = 64, n_predicted_features = 7):
        self.n_timesteps = n_timesteps
        self.n_features = n_features
        self.neurons = neurons
        self.activation = activation
        self.optimizer = optimizer
        self.loss = loss
        self.metrics = metrics
        self.dropout = dropout
        self.conv_layer = conv_layer
        self.nfilters = nfilters
        self.conv_act = conv_act
        self.pool_size = pool_size
        self.classification = classification
        self.n_predicted_timesteps = n_predicted_timesteps
        self.epochs = epochs
        self.batch  = batch
        self.n_predicted_features = n_predicted_features
        
    def create_model(self):

        model = lstm(neurons=self.neurons, 
                     activation_func = self.activation, 
                     nTimesteps = self.n_timesteps, 
                     nFeatures = self.n_features, 
                     npredTimesteps = 1, 
                     npredFeatures = self.n_predicted_features, 
                     model_optimizer = self.optimizer, 
                     model_loss = self.loss, 
                     model_metrics = self.metrics, 
                     dropout=self.dropout, 
                     conv_layer=self.conv_layer, 
                     nfilters=self.nfilters, 
                     cact =self.conv_act, 
                     cpool=2,
                     classify=self.classification)
        return model
    

    def fit(self, X, y):
        if self.classification:
            self.classes_ = np.unique(y)
        # print(f'shape of Y: {y.shape}')
        self.model = self.create_model()
        self.model.fit(X, y, nEpochs = 1, nBatches= 64, val_split = 0.2, verb = 2, shuf = False)

        return self

    def predict(self, X):
        # Make predictions
        # print(f'shape of x (predict): {X.shape}')
        y_pred = self.model.predict(X)
        # print(f'shape of y_pred: {y_pred.shape}')
        if self.classification:
            y_pred = (y_pred >= 0.5).astype(float)
        # print(y_pred[0][0])
        return y_pred 

In [7]:
def custom_mse(y_true, y_pred):
    # print(f"Shape of y_true in custom scoring function: {y_true.shape}")
    # print(y_true[0][0])
    # print(f"Shape of y_pred in custom scoring function: {y_pred.shape}")
    # print(y_pred[0][0])

    # # flatten
    y_true_2d = y_true.reshape(-1, y_true.shape[2])  # Shape becomes (2148, 9)
    y_pred_2d = y_pred.reshape(-1, y_pred.shape[2])  # Shape becomes (2148, 9)

    y_true_2d = y_true_2d.astype(int)
    y_pred_2d = y_pred_2d.astype(int)

    # print(f'shape of true and pred after reshape: {y_true_2d.shape} and {y_pred_2d.shape}')
    # print(f'type: {type(y_true_2d[0][0])}')
    # print(f"Data type of y_true: {y_true_2d.dtype}")
    # print(f"Data type of y_pred: {y_pred_2d.dtype}")

    mse = -np.mean((y_true - y_pred) ** 2)

    return mse

In [8]:
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier  # Example model
from scipy.stats import randint, uniform  # For defining distributions
from sklearn.metrics import make_scorer, mean_squared_error


tscv = TimeSeriesSplit(n_splits = 3)

### 25 npast_timesteps
- Timesteps tried are defined outside of the RandomizedSearch to make preprocessing easier and avoid nan scoring values
- Tried to run 25, 50, 75 timesteps using for loop and list, but kernal died or randomized search terminated due to memory

In [9]:
past_timepoints = [25]
conv_layer = [False, True]

best_params = []
best_score = []

for i in past_timepoints:
    # reset x and y
    X = np.array(data[numerical].drop(columns=['Time']))
    Y = np.array(data[numerical].drop(columns=['Time']))

    # preprocess data (2D)
    scalerX, X_norm = utils.scale(X)
    scalerY, Y_norm = utils.scale(Y)
    # print(f'x_shape scaled: {X_norm.shape}')
    # print(f'y_shape scaled: {Y_norm.shape}')

    # Reshape based on timesteps
    [X, _] = utils.prep_lstm_data(X_norm, i, 60) 
    [_, Y] = utils.prep_lstm_data(Y_norm, i, 60) 

    # print(f'shape of y: {Y.shape}')
    for conv in conv_layer: 
        # will execute with conv_layer False, then conv_layer True, 
        # must be in this order to prevent input shape error
        # The data will reset to 2D when new npast_timesteps in the outter for loop
        
        if conv: # reshape input
            X = X.reshape((X.shape[0], 1, X.shape[1], X.shape[2]))
            
        # define parameters to gridsearch
        param_dict = {
        'neurons': [[32, 32], [64, 32], [128, 64, 32]],
        'activation':["tanh", 'relu'],
        'n_timesteps':[i],
        'n_features':[7],
        'n_predicted_timesteps': [60],
        'n_predicted_features' : [7],
        'optimizer': ["adam"],
        'loss': ["mse"],
        'metrics': [["mse", 'mae']],
        'dropout':[0.1, 0.2, 0.3],
        'conv_layer':[conv],
        'nfilters':[32, 64], # 32, 64
        'conv_act': ['relu', 'tanh'],
        'pool_size':[2],
        'classification': [False],
        'epochs':[10, 50], # 50
        'batch':[64],
        }
        # print(Y.shape)
        # print(type(Y[0][0][0]))


        # print(Y.shape)
        # print(type(Y[0][0][0]))

        # Random Search
        random_search = RandomizedSearchCV(
            estimator= PreprocessingWrapper(),
            param_distributions=param_dict,
            n_iter=20,  # Number of different combinations to sample
            scoring = make_scorer(custom_mse), 
            cv=tscv,  # Number of cross-validation folds
            verbose=1,  # Display search progress
            n_jobs= 1,  
            random_state=42  # For reproducibility
        )

        random_search.fit(X, Y)
        best_params.append(random_search.best_params_)
        best_score.append(random_search.best_score_)

    


Fitting 3 folds for each of 20 candidates, totalling 60 fits


2024-12-11 21:40:25.763550: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


14/14 - 3s - 190ms/step - loss: 0.2038 - mae: 0.3315 - mse: 0.2038 - val_loss: 0.1355 - val_mae: 0.2569 - val_mse: 0.1355
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
27/27 - 3s - 103ms/step - loss: 0.1546 - mae: 0.2819 - mse: 0.1546 - val_loss: 0.0806 - val_mae: 0.2119 - val_mse: 0.0806
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
40/40 - 3s - 76ms/step - loss: 0.1415 - mae: 0.2792 - mse: 0.1415 - val_loss: 0.0841 - val_mae: 0.2236 - val_mse: 0.0841
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
14/14 - 3s - 203ms/step - loss: 0.1360 - mae: 0.2721 - mse: 0.1360 - val_loss: 0.0779 - val_mae: 0.2162 - val_mse: 0.0779
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
27/27 - 3s - 101ms/step - loss: 0.1248 - mae: 0.2626 - mse: 0.1248 - val_loss: 0.0604 - val_mae: 0.1824 - val_mse: 0.0604
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
40/40 - 3s - 75ms/step - loss: 0.0969 - mae: 0.2382 - mse: 0.0969 - val_loss: 0.0642 - val_mae: 0.1939 - val_mse: 0.0642
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
14/14 - 2s - 144ms/step - loss

/home/kxlee/miniconda3/envs/msse-python/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


53/53 - 7s - 126ms/step - loss: 0.0836 - mae: 0.2203 - mse: 0.0836 - val_loss: 0.0850 - val_mae: 0.2350 - val_mse: 0.0850
Fitting 3 folds for each of 20 candidates, totalling 60 fits
14/14 - 5s - 372ms/step - loss: 0.2021 - mae: 0.3199 - mse: 0.2021 - val_loss: 0.1186 - val_mae: 0.2491 - val_mse: 0.1186
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
27/27 - 4s - 136ms/step - loss: 0.1406 - mae: 0.2764 - mse: 0.1406 - val_loss: 0.0806 - val_mae: 0.2057 - val_mse: 0.0806
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
40/40 - 4s - 94ms/step - loss: 0.1377 - mae: 0.2663 - mse: 0.1377 - val_loss: 0.0858 - val_mae: 0.2174 - val_mse: 0.0858
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
14/14 - 4s - 274ms/step - loss: 0.1899 - mae: 0.3206 - mse: 0.1899 - val_loss: 0.1034 - val_mae: 0.2390 - val_mse: 0.1034
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
27/27 - 4s - 131ms/step - loss: 0.1496 - mae: 0.2762 - mse: 0.1496 - val_loss: 0.0607 - val_mae: 0.1790 - val_mse: 0.0607
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
40/40 - 

In [10]:
# # take the best score and find associated parameters

max_index = best_score.index(max(best_score))
print(f'Best Params for 25 npast_timesteps: {best_params[max_index]}')
print(f'Best Score for 25 npast_timesteps: {max(best_score)}')

Best Params for 25 npast_timesteps: {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 64, 'neurons': [128, 64, 32], 'n_timesteps': 25, 'n_predicted_timesteps': 60, 'n_predicted_features': 7, 'n_features': 7, 'metrics': ['mse', 'mae'], 'loss': 'mse', 'epochs': 10, 'dropout': 0.3, 'conv_layer': False, 'conv_act': 'tanh', 'classification': False, 'batch': 64, 'activation': 'tanh'}
Best Score for 25 npast_timesteps: -0.07911169500806445


In [ ]:
Best Params for 25 npast_timesteps: {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 64, 'neurons': [128, 64, 32], 'n_timesteps': 25, 'n_predicted_timesteps': 60, 'n_predicted_features': 7, 'n_features': 7, 'metrics': ['mse', 'mae'], 'loss': 'mse', 'epochs': 10, 'dropout': 0.3, 'conv_layer': False, 'conv_act': 'tanh', 'classification': False, 'batch': 64, 'activation': 'tanh'}
Best Score for 25 npast_timesteps: -0.07911169500806445

In [11]:
# append to list
best_scores_ = []
best_params_ = []

best_scores_.append(max(best_score))
best_params_.append(best_params[max_index])


### 50 npast_timesteps

In [9]:
past_timepoints = [50]
conv_layer = [False, True]

best_params_50 = []
best_score_50 = []

for i in past_timepoints:
    # reset x and y
    X = np.array(data[numerical].drop(columns=['Time']))
    Y = np.array(data[numerical].drop(columns=['Time']))

    # preprocess data (2D)
    scalerX, X_norm = utils.scale(X)
    scalerY, Y_norm = utils.scale(Y)
    # print(f'x_shape scaled: {X_norm.shape}')
    # print(f'y_shape scaled: {Y_norm.shape}')

    # Reshape based on timesteps
    [X, _] = utils.prep_lstm_data(X_norm, i, 60) 
    [_, Y] = utils.prep_lstm_data(Y_norm, i, 60) 

    # print(f'shape of y: {Y.shape}')
    for conv in conv_layer: 
        # will execute with conv_layer False, then conv_layer True, 
        # must be in this order to prevent input shape error
        # The data will reset to 2D when new npast_timesteps in the outter for loop
        
        if conv: # reshape input
            X = X.reshape((X.shape[0], 1, X.shape[1], X.shape[2]))
            
        # define parameters to gridsearch
        param_dict = {
        'neurons': [[32, 32], [64, 32], [128, 64, 32]],
        'activation':["tanh", 'relu'],
        'n_timesteps':[i],
        'n_features':[7],
        'n_predicted_timesteps': [60],
        'n_predicted_features' : [7],
        'optimizer': ["adam"],
        'loss': ["mse"],
        'metrics': [["mse", 'mae']],
        'dropout':[0.1, 0.2, 0.3],
        'conv_layer':[conv],
        'nfilters':[32, 64], # 32, 64
        'conv_act': ['relu', 'tanh'],
        'pool_size':[2],
        'classification': [False],
        'epochs':[10, 50], # 50
        'batch':[64],
        }
        # print(Y.shape)
        # print(type(Y[0][0][0]))


        # print(Y.shape)
        # print(type(Y[0][0][0]))

        # Random Search
        random_search = RandomizedSearchCV(
            estimator= PreprocessingWrapper(),
            param_distributions=param_dict,
            n_iter=20,  # Number of different combinations to sample
            scoring = make_scorer(custom_mse), 
            cv=tscv,  # Number of cross-validation folds
            verbose=2,  # Display search progress
            n_jobs=1,  # Use all available CPUs
            random_state=42  # For reproducibility
        )

        random_search.fit(X, Y)
        best_params_50.append(random_search.best_params_)
        best_score_50.append(random_search.best_score_)

    


Fitting 3 folds for each of 20 candidates, totalling 60 fits


2024-12-11 21:51:24.810678: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


14/14 - 3s - 215ms/step - loss: 0.1984 - mae: 0.3259 - mse: 0.1984 - val_loss: 0.1310 - val_mae: 0.2700 - val_mse: 0.1310
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
[CV] END activation=relu, batch=64, classification=False, conv_act=tanh, conv_layer=False, dropout=0.1, epochs=50, loss=mse, metrics=['mse', 'mae'], n_features=7, n_predicted_features=7, n_predicted_timesteps=60, n_timesteps=50, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   4.1s
27/27 - 4s - 131ms/step - loss: 0.1348 - mae: 0.2643 - mse: 0.1348 - val_loss: 0.0678 - val_mae: 0.1929 - val_mse: 0.0678
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
[CV] END activation=relu, batch=64, classification=False, conv_act=tanh, conv_layer=False, dropout=0.1, epochs=50, loss=mse, metrics=['mse', 'mae'], n_features=7, n_predicted_features=7, n_predicted_timesteps=60, n_timesteps=50, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   4.8s
40/40 - 6s - 151ms/step - loss: 0.1105 - mae: 0.2417 - mse: 

/home/kxlee/miniconda3/envs/msse-python/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


53/53 - 8s - 152ms/step - loss: 0.0781 - mae: 0.2098 - mse: 0.0781 - val_loss: 0.0764 - val_mae: 0.2310 - val_mse: 0.0764
Fitting 3 folds for each of 20 candidates, totalling 60 fits
14/14 - 7s - 473ms/step - loss: 0.1645 - mae: 0.2959 - mse: 0.1645 - val_loss: 0.1115 - val_mae: 0.2328 - val_mse: 0.1115
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
[CV] END activation=relu, batch=64, classification=False, conv_act=tanh, conv_layer=True, dropout=0.1, epochs=50, loss=mse, metrics=['mse', 'mae'], n_features=7, n_predicted_features=7, n_predicted_timesteps=60, n_timesteps=50, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   7.8s
27/27 - 4s - 142ms/step - loss: 0.1445 - mae: 0.2808 - mse: 0.1445 - val_loss: 0.0744 - val_mae: 0.1986 - val_mse: 0.0744
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
[CV] END activation=relu, batch=64, classification=False, conv_act=tanh, conv_layer=True, dropout=0.1, epochs=50, loss=mse, metrics=['mse', 'mae'], n_features=7, n_predicted_features=7, 

In [10]:
# take the best score and find associated parameters

max_index_50 = best_score_50.index(max(best_score_50))
print(f'Best Params for 50 npast_timesteps: {best_params_50[max_index_50]}')
print(f'Best Scorefor 50 npast_timesteps: {max(best_score_50)}')

Best Params for 50 npast_timesteps: {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 64, 'neurons': [64, 32], 'n_timesteps': 50, 'n_predicted_timesteps': 60, 'n_predicted_features': 7, 'n_features': 7, 'metrics': ['mse', 'mae'], 'loss': 'mse', 'epochs': 10, 'dropout': 0.2, 'conv_layer': True, 'conv_act': 'tanh', 'classification': False, 'batch': 64, 'activation': 'tanh'}
Best Scorefor 50 npast_timesteps: -0.07934705068987576


Best Params for 50 npast_timesteps: {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 64, 'neurons': [64, 32], 'n_timesteps': 50, 'n_predicted_timesteps': 60, 'n_predicted_features': 7, 'n_features': 7, 'metrics': ['mse', 'mae'], 'loss': 'mse', 'epochs': 10, 'dropout': 0.2, 'conv_layer': True, 'conv_act': 'tanh', 'classification': False, 'batch': 64, 'activation': 'tanh'}
Best Scorefor 50 npast_timesteps: -0.07934705068987576

In [11]:
best_scores_.append(max(best_score_50))
best_params_.append(best_params_50[max_index_50])

NameError: name 'best_scores_' is not defined

### 75 npast_timesteps

In [9]:
past_timepoints = [75]
conv_layer = [False, True]

best_params_75 = []
best_score_75 = []

for i in past_timepoints:
    # reset x and y
    X = np.array(data[numerical].drop(columns=['Time']))
    Y = np.array(data[numerical].drop(columns=['Time']))

    # preprocess data (2D)
    scalerX, X_norm = utils.scale(X)
    scalerY, Y_norm = utils.scale(Y)
    # print(f'x_shape scaled: {X_norm.shape}')
    # print(f'y_shape scaled: {Y_norm.shape}')

    # Reshape based on timesteps
    [X, _] = utils.prep_lstm_data(X_norm, i, 60) 
    [_, Y] = utils.prep_lstm_data(Y_norm, i, 60) 

    # print(f'shape of y: {Y.shape}')
    for conv in conv_layer: 
        # will execute with conv_layer False, then conv_layer True, 
        # must be in this order to prevent input shape error
        # The data will reset to 2D when new npast_timesteps in the outter for loop
        
        if conv: # reshape input
            X = X.reshape((X.shape[0], 1, X.shape[1], X.shape[2]))
            
        # define parameters to gridsearch
        param_dict = {
        'neurons': [[32, 32], [64, 32], [128, 64, 32]],
        'activation':["tanh", 'relu'],
        'n_timesteps':[i],
        'n_features':[7],
        'n_predicted_timesteps': [60],
        'n_predicted_features' : [7],
        'optimizer': ["adam"],
        'loss': ["mse"],
        'metrics': [["mse", 'mae']],
        'dropout':[0.1, 0.2, 0.3],
        'conv_layer':[conv],
        'nfilters':[32, 64], # 32, 64
        'conv_act': ['relu', 'tanh'],
        'pool_size':[2],
        'classification': [False],
        'epochs':[10, 50], # 50
        'batch':[64],
        }
        # print(Y.shape)
        # print(type(Y[0][0][0]))


        # print(Y.shape)
        # print(type(Y[0][0][0]))

        # Random Search
        random_search = RandomizedSearchCV(
            estimator= PreprocessingWrapper(),
            param_distributions=param_dict,
            n_iter=20,  # Number of different combinations to sample
            scoring = make_scorer(custom_mse), 
            cv=tscv,  # Number of cross-validation folds
            verbose=2,  # Display search progress
            n_jobs= 1,  # Use all available CPUs
            random_state=42  # For reproducibility
        )

        random_search.fit(X, Y)
        best_params_75.append(random_search.best_params_)
        best_score_75.append(random_search.best_score_)

    


Fitting 3 folds for each of 20 candidates, totalling 60 fits


2024-12-11 22:04:58.003187: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


14/14 - 3s - 234ms/step - loss: 0.2191 - mae: 0.3460 - mse: 0.2191 - val_loss: 0.1570 - val_mae: 0.2761 - val_mse: 0.1570
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
[CV] END activation=relu, batch=64, classification=False, conv_act=tanh, conv_layer=False, dropout=0.1, epochs=50, loss=mse, metrics=['mse', 'mae'], n_features=7, n_predicted_features=7, n_predicted_timesteps=60, n_timesteps=75, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   4.4s
27/27 - 4s - 151ms/step - loss: 0.1373 - mae: 0.2679 - mse: 0.1373 - val_loss: 0.0620 - val_mae: 0.1808 - val_mse: 0.0620
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
[CV] END activation=relu, batch=64, classification=False, conv_act=tanh, conv_layer=False, dropout=0.1, epochs=50, loss=mse, metrics=['mse', 'mae'], n_features=7, n_predicted_features=7, n_predicted_timesteps=60, n_timesteps=75, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   5.1s
40/40 - 5s - 126ms/step - loss: 0.1284 - mae: 0.2531 - mse: 

/home/kxlee/miniconda3/envs/msse-python/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


53/53 - 10s - 183ms/step - loss: 0.1016 - mae: 0.2385 - mse: 0.1016 - val_loss: 0.0780 - val_mae: 0.2211 - val_mse: 0.0780
Fitting 3 folds for each of 20 candidates, totalling 60 fits
14/14 - 5s - 356ms/step - loss: 0.1912 - mae: 0.3344 - mse: 0.1912 - val_loss: 0.1206 - val_mae: 0.2562 - val_mse: 0.1206
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
[CV] END activation=relu, batch=64, classification=False, conv_act=tanh, conv_layer=True, dropout=0.1, epochs=50, loss=mse, metrics=['mse', 'mae'], n_features=7, n_predicted_features=7, n_predicted_timesteps=60, n_timesteps=75, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   6.2s
27/27 - 4s - 155ms/step - loss: 0.1490 - mae: 0.2838 - mse: 0.1490 - val_loss: 0.0748 - val_mae: 0.1974 - val_mse: 0.0748
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
[CV] END activation=relu, batch=64, classification=False, conv_act=tanh, conv_layer=True, dropout=0.1, epochs=50, loss=mse, metrics=['mse', 'mae'], n_features=7, n_predicted_features=7,

In [10]:
# take the best score and find associated parameters

max_index_75 = best_score_75.index(max(best_score_75))
print(f'Best Params for 75 npast_timesteps: {best_params_75[max_index_75]}')
print(f'Best Scorefor 75 npast_timesteps: {max(best_score_75)}')

Best Params for 75 npast_timesteps: {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 64, 'neurons': [64, 32], 'n_timesteps': 75, 'n_predicted_timesteps': 60, 'n_predicted_features': 7, 'n_features': 7, 'metrics': ['mse', 'mae'], 'loss': 'mse', 'epochs': 10, 'dropout': 0.2, 'conv_layer': True, 'conv_act': 'tanh', 'classification': False, 'batch': 64, 'activation': 'tanh'}
Best Scorefor 75 npast_timesteps: -0.07810362410263826


### Best Parameters for Regression

Add manually to list becuase kernal crashed

In [12]:
scores = [-0.07911169500806445, -0.07934705068987576, -0.07810362410263826]
params = [{'pool_size': 2, 'optimizer': 'adam', 'nfilters': 64, 'neurons': [128, 64, 32], 'n_timesteps': 25, 'n_predicted_timesteps': 60, 'n_predicted_features': 7, 'n_features': 7, 'metrics': ['mse', 'mae'], 'loss': 'mse', 'epochs': 10, 'dropout': 0.3, 'conv_layer': False, 'conv_act': 'tanh', 'classification': False, 'batch': 64, 'activation': 'tanh'}, 
          {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 64, 'neurons': [64, 32], 'n_timesteps': 50, 'n_predicted_timesteps': 60, 'n_predicted_features': 7, 'n_features': 7, 'metrics': ['mse', 'mae'], 'loss': 'mse', 'epochs': 10, 'dropout': 0.2, 'conv_layer': True, 'conv_act': 'tanh', 'classification': False, 'batch': 64, 'activation': 'tanh'},
            {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 64, 'neurons': [64, 32], 'n_timesteps': 75, 'n_predicted_timesteps': 60, 'n_predicted_features': 7, 'n_features': 7, 'metrics': ['mse', 'mae'], 'loss': 'mse', 'epochs': 10, 'dropout': 0.2, 'conv_layer': True, 'conv_act': 'tanh', 'classification': False, 'batch': 64, 'activation': 'tanh'}
]
max_index = scores.index(max(scores))
print(min(scores))
print(params[max_index])


-0.07934705068987576
{'pool_size': 2, 'optimizer': 'adam', 'nfilters': 64, 'neurons': [64, 32], 'n_timesteps': 75, 'n_predicted_timesteps': 60, 'n_predicted_features': 7, 'n_features': 7, 'metrics': ['mse', 'mae'], 'loss': 'mse', 'epochs': 10, 'dropout': 0.2, 'conv_layer': True, 'conv_act': 'tanh', 'classification': False, 'batch': 64, 'activation': 'tanh'}
